# SpaceX Falcon 9 Landing Prediction

# Interactive Visual Analytics with Folium

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis notebook, we have visualized the SpaceX launch dataset using matplotlib and seaborn and discovered some preliminary correlations between the launch site and success rates. In this lab, we will be performing more interactive visual analytics using Folium.

## Objectives

This notebook contains the following tasks:

1. Task 1: Mark all launch sites on a map
2. Task 2: Mark the success/failed launches for each site on the map
3. Task 3: Calculate the distances between a launch site to its proximities

After completed the above tasks, we should be able to find some geographical patterns about launch sites.

## Import Libraries and Define Auxiliary Functions

In [1]:
# Install folium library

!pip install folium

In [2]:
# Import necessary libraries

import folium
import pandas as pd

In [3]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map

In [4]:
# Read the 'spacex_launch_geo.csv'

spacex_data = pd.read_csv('spacex_launch_geo.csv')

In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`

spacex_data = spacex_data[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_data = spacex_data.groupby(['Launch Site'], as_index = False).first()
launch_sites_data = launch_sites_data[['Launch Site', 'Lat', 'Long']]
launch_sites_data

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [6]:
# Start location is NASA Johnson Space Center

nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location = nasa_coordinate, zoom_start = 10)

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name

circle = folium.Circle(nasa_coordinate, radius = 1000, color = '#d35400', fill = True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon = DivIcon(
        icon_size = (20,20),
        icon_anchor = (0,0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [8]:
# Initial the map

site_map = folium.Map(location = nasa_coordinate, zoom_start = 5)
for launch_site, site_lat, site_long in zip(launch_sites_data['Launch Site'], launch_sites_data['Lat'], launch_sites_data['Long']):
    site_coordinate = [site_lat, site_long]
    circle = folium.Circle(site_coordinate, radius = 1000, color = '#d35400', fill = True).add_child(folium.Popup(launch_site))
    marker = folium.map.Marker(
        site_coordinate,
        icon = DivIcon(
            icon_size = (20, 20),
            icon_anchor = (0, 0),
            html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site,
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

Explanation:

1. Most of Launch sites considered in this project are in proximity to the Equator line. Launch sites are made at the closest point possible to Equator line, because anything on the surface of the Earth at the equator is already moving at the maximum speed (1670 kilometers per hour). For example launching from the equator makes the spacecraft move almost 500 km/hour faster once it is launched compared half way to north pole.
2. All launch sites considered in this project are in very close proximity to the coast While starting rockets towards the ocean we minimise the risk of having any debris dropping or exploding near people.

## Task 2: Mark the success/failed launches for each site on the map

In [9]:
# Recall spacex_data

spacex_data.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [10]:
# Create a MarkerCluster object

marker_cluster = MarkerCluster()

In [11]:
# Apply a function to check the value of `class` column

spacex_data['marker_color'] = list(map(lambda x: 'green' if x == 1 else 'red', spacex_data['class']))
spacex_data.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [12]:
# Function to assign color to launch outcome

def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_data['marker_color'] = spacex_data['class'].apply(assign_marker_color)
spacex_data.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [13]:
# Add the Marker cluster to the site map

site_map.add_child(marker_cluster)
for site_lat, site_long, marker_color in zip(spacex_data['Lat'], spacex_data['Long'], spacex_data['marker_color']):
    site_coordinate = [site_lat, site_long]
    marker = folium.map.Marker(
        site_coordinate,
        icon = folium.Icon(color = 'white', 
                         icon_color = marker_color)
    )
    marker.add_to(marker_cluster)
site_map

## Task 3: Calculate the distances between a launch site to its proximities

In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map

formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position = 'topright',
    separator = ' Long: ',
    empty_string = 'NaN',
    lng_first = False,
    num_digits = 20,
    prefix = 'Lat:',
    lat_formatter = formatter,
    lng_formatter = formatter,
)

site_map.add_child(mouse_position)
site_map

In [15]:
# Calculate the distance between two points on the map based on their Lat and Long values

from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [16]:
# distance_railway = calculate_distance(lat1, lon1, lat2, lon2)

railway_marker = [28.55752, -80.80155]
launch_coordinate = [28.57337, -80.64669]
distance_railway = calculate_distance(railway_marker[0], railway_marker[1], launch_coordinate[0], launch_coordinate[1])
distance_railway # distance in km

15.230651245141603

In [17]:
# create and add a folium.Marker on your selected closest raiwaly point on the map

# show the distance to the launch site using the icon property 
marker = folium.map.Marker(
        railway_marker,
        icon = DivIcon(
            icon_size = (400, 400),
            icon_anchor = (0, 0),
            html = '<div style="font-size:400; color:#0c10f2;"><b>%s</b></div>' % str(round(distance_railway, 2))+' km',
            )
    )
marker.add_to(site_map)
site_map

In [18]:
# Create a `folium.PolyLine` object using the railway point coordinate and launch site coordinate

folium.PolyLine([railway_marker, launch_coordinate], color = 'blue').add_to(site_map)
site_map

In [19]:
# Create a marker with distance to a closest city, coastline, highway, etc.

# Draw a line between the marker to the launch site
city   = [28.61200, -80.80788]
coastline = [28.5858, -80.79952]
highway   = [28.5402, -80.85079]

city_distance = calculate_distance(city[0], city[1], launch_coordinate[0], launch_coordinate[1])
coastline_distance = calculate_distance(coastline[0], coastline[1], launch_coordinate[0], launch_coordinate[1])
highway_distance = calculate_distance(highway[0], highway[1], launch_coordinate[0], launch_coordinate[1])

colors = ['red','orange','green']
html_colors = ['#dc3545','#fd7e14','#198754']

for coordinate ,distance, color, html_color in zip([city, coastline, highway], [city_distance, coastline_distance, highway_distance], colors, html_colors):
    marker = folium.map.Marker(
            coordinate,
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html='<div style="font-size: 12; color:'+html_color+';"><b>%s</b></div>' % str(round(distance, 2)) + 'km',
                )
            )
    marker.add_to(site_map)
    folium.PolyLine([coordinate, launch_coordinate], color = color).add_to(site_map)
site_map

Explanation:

1. From the visual analysis of the launch site KSC LC-39A we can clearly see that it is
    a. relative close to railway (15.23 km)
    b. relative close to highway (20.28 km)
    c. relative close to coastline (14.99 km)
2. Also the launch site KSC LC-39A is relative close to its closest city Titusville (16.32 km)
3. Failed rocket with its high speed can cover distances like 15-20 km in few seconds. It could be potentially dangerous to populated areas

## -END-